In [1]:
from bs4 import BeautifulSoup, NavigableString, Tag
import requests
import pandas as pd

In [2]:
global df
df=pd.DataFrame(columns=['Amenities','Bedrooms','Bathrooms','Area (m²)',
 'Furnished',
 'Level',
 'Ad Type',
 'Type',
 'Payment Option',
 'price',
 'title',
 'location',
 'Compound'])
#df=df.append(row, ignore_index=True)
#df

In [3]:

#import pandas
#import numpy

def GetHouseList(pages_from,pages_to):
    global df
    data=[]
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:79.0) Gecko/20100101 Firefox/79.0'}
    #df=pandas.DataFrame(columns=['title','price','location','property_type','bedrooms','bathrooms','area'])
    for pageNo in range(pages_from,pages_to):
        #url="https://www.olx.com.eg/en/properties/apartments-duplex-for-sale/cairo/?search%5Bfilter_enum_ad_type%5D%5B0%5D=1&search%5Bfilter_enum_type%5D%5B0%5D=1&search%5Bfilter_enum_payment_options%5D%5B0%5D=1&page={}".format(pageNo+1)
        # Url for Gardenia
        url="https://www.olx.com.eg/en/properties/apartments-duplex-for-sale/cairo/q-%D8%AC%D8%A7%D8%B1%D8%AF%D9%8A%D9%86%D9%8A%D8%A7-%D8%A7%D9%84%D8%AA%D8%AC%D9%85%D8%B9-%D8%A7%D9%84%D8%AE%D8%A7%D9%85%D8%B3/?page={}".format(pageNo+1)
        print(pageNo+1,df.shape[0])
        r = requests.get(url,headers=headers)
        content = r.content
        soup = BeautifulSoup(content,features="lxml")
        #print(soup)
        features={}
        for d in soup.findAll('div', attrs={'class':'ads__item'}):
            features={}
            try:
                 #get details
                detailsUrl= d.find('a', attrs={'class':'ads__item__ad--title'},href=True)
                #print(detailsUrl['href'])
                detailsUrl=detailsUrl['href'].replace('https://www.olx.com.eg/','https://www.olx.com.eg/en/')
                detialRequest = requests.get(detailsUrl,headers=headers)
                detailContent = detialRequest.content
                detailSoup = BeautifulSoup(detailContent,features="lxml")
                table_tr = detailSoup.find("table", attrs={ "class" : "details fixed marginbott20 margintop5 full" }).find_all("tr")
                for tr in table_tr:
                    for td in tr:
                        if isinstance(td, NavigableString):
                            continue
                        if isinstance(td, Tag) and td.find('tr')!=None :
                            #print(body_child.name)
                            name=td.find('tr').find('th').text.strip()
                            value=td.find('tr').find('td').text.strip()
                            features[name]=value.replace('\t','').replace('\n\n',',')
                ads_info=d.find('div', attrs={'class':'ads__item__info'})           
                if ads_info.find('p',{'class':'ads__item__price price'}) ==None:
                    features['price'] = ads_info.find('p',{'class':'ads__item__price price price-arranged'}).text.replace('\t','').replace('\n','')
                else:
                    features['price'] = ads_info.find('p',{'class':'ads__item__price price'}).text.replace('\t','').replace('\n','')
                features['title'] = ads_info.find('a', attrs={'class':'ads__item__ad--title'}).text.replace('\t','').replace('\n','')
                features['location']  = ads_info.find('p', attrs={'class':'ads__item__location'}).text.replace('\t','').replace('\n','')
                df=df.append(features,ignore_index=True)
                #pd.concat(df,df_new)
                #print(features)
                #break
            except Exception as ex :
                print(ex)
                continue
    return features


In [4]:
GetHouseList(0,8)
df.to_csv('./Data/olx_data_Gardenia.csv',index=False)

1 0
2 47
3 94
4 141
5 188
6 235
7 258
8 297


In [71]:
df=pd.read_csv('./Data/olx_data_Gardenia.csv')

In [72]:
df.head()

,Amenities,Bedrooms,Bathrooms,Area (m²),Furnished,Level,Ad Type,Type,Payment Option,price,title,location,Compound
0,"Pets Allowed,Electricity Meter,Landline",3,2,220,NaN,2,For Sale,Apartment,Cash,"1,300,000 EGP",شقة للبيع 200 متر هااى لوكس بسعر لقطة,New Cairo - El Tagamoa,Abu Al-Hool
1,"Balcony,Electricity Meter,Water Meter,Natural Gas",2,1,110,No,1,For Sale,Apartment,Cash,"600,000 EGPnegotiable",ارخص شقه للبيع جاردنيا 1 بالتجمع الخامس اس...,New Cairo - El Tagamoa,Abu Al-Hool
2,"Balcony,Pets Allowed,Electricity Meter,Water M...",2,2,145,No,2,For Sale,Apartment,Cash,"1,600,000 EGPnegotiable",شقة 145 متر بفيلات الحى الاول منطقة مميزة ...,New Cairo - El Tagamoa,Abu Al-Hool
3,"Balcony,Covered Parking,Pets Allowed,Electrici...",3,2,160,No,4,For Sale,Apartment,Cash,"1,650,000 EGPnegotiable",شقة 160 م عمارات الحى الثالث بجانب فاطمة ا...,New Cairo - El Tagamoa,Abu Al-Hool
4,NaN,3,2,220,NaN,Ground,For Sale,Duplex,Cash or Installment,"675,000 EGP",دوبلكس للبيع 220متر في التجمع الخامس,New Cairo - El Tagamoa,Abu Al-Hool


In [73]:
df.Compound='Abu Al-Hool'
df.head()

,Amenities,Bedrooms,Bathrooms,Area (m²),Furnished,Level,Ad Type,Type,Payment Option,price,title,location,Compound
0,"Pets Allowed,Electricity Meter,Landline",3,2,220,NaN,2,For Sale,Apartment,Cash,"1,300,000 EGP",شقة للبيع 200 متر هااى لوكس بسعر لقطة,New Cairo - El Tagamoa,Abu Al-Hool
1,"Balcony,Electricity Meter,Water Meter,Natural Gas",2,1,110,No,1,For Sale,Apartment,Cash,"600,000 EGPnegotiable",ارخص شقه للبيع جاردنيا 1 بالتجمع الخامس اس...,New Cairo - El Tagamoa,Abu Al-Hool
2,"Balcony,Pets Allowed,Electricity Meter,Water M...",2,2,145,No,2,For Sale,Apartment,Cash,"1,600,000 EGPnegotiable",شقة 145 متر بفيلات الحى الاول منطقة مميزة ...,New Cairo - El Tagamoa,Abu Al-Hool
3,"Balcony,Covered Parking,Pets Allowed,Electrici...",3,2,160,No,4,For Sale,Apartment,Cash,"1,650,000 EGPnegotiable",شقة 160 م عمارات الحى الثالث بجانب فاطمة ا...,New Cairo - El Tagamoa,Abu Al-Hool
4,NaN,3,2,220,NaN,Ground,For Sale,Duplex,Cash or Installment,"675,000 EGP",دوبلكس للبيع 220متر في التجمع الخامس,New Cairo - El Tagamoa,Abu Al-Hool


In [74]:
df.to_csv('./Data/olx_data_Gardenia.csv',index=False)

In [75]:
# read all data
df_all=pd.read_csv('./Data/olx_data_all.csv')
df_all.shape,df.shape

((26500, 13), (350, 13))

In [76]:
# check if there are appartments in gardenia but compound is null
df_all.loc[(df_all.title.str.contains('جاردينيا') | df_all.Compound.str.contains('Gardenia',na=False)) ,'Compound']='Abu Al-Hool'
df_all[df_all.title.str.contains('جاردينيا')]

,Amenities,Bedrooms,Bathrooms,Area (m²),Furnished,Level,Ad Type,Type,Payment Option,price,title,location,Compound
56,NaN,3,2,240,NaN,3,For Sale,Apartment,Cash,"1,000,000 EGP",دور كامل على فاصل رئيسى فى جاردينيا هايتس ...,New Cairo - El Tagamoa,Abu Al-Hool
553,"Balcony,Electricity Meter",2,1,100,No,3,For Sale,Apartment,Cash,"5,500 EGPnegotiable",شقة ب جاردينيا هايتس 2- ابوالهول 2 سابقا,New Cairo - El Tagamoa,Abu Al-Hool
955,Electricity Meter,2,1,110,No,Ground,For Sale,Apartment,Cash,"650,000 EGP",شقة مع هدية و مفاجأة فى جاردينيا هايتس دور...,New Cairo - El Tagamoa,Abu Al-Hool
1437,"Electricity Meter,Landline,Elevator",2,1,130,NaN,2,For Sale,Apartment,Cash,"885,000 EGP",شقة للبيع بجاردينيا هايتس 1,New Cairo - El Tagamoa,Abu Al-Hool
1493,Balcony,3,2,170,No,3,For Sale,Apartment,Cash,"860,000 EGP",شقه 170م بجاردينيا هايتس استلام فوووووورى,New Cairo - El Tagamoa,Abu Al-Hool
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23880,NaN,3,2,300,No,3,For Sale,Apartment,Cash,"4,500,000 EGP",عمارة للبيع بجاردينيا,Zahraa Al Maadi,Abu Al-Hool
24034,NaN,3,2,230,No,1,For Sale,Apartment,Cash,"1,050,000 EGPnegotiable",دور كامل فى جاردينيا 1 بسعر محروق,New Cairo - El Tagamoa,Abu Al-Hool
24670,NaN,3,2,135,NaN,3,For Sale,Apartment,Cash,"450,000 EGP",الحق شقة ١٣٥م دور ٣ بحى جاردينيا بعداد الك...,Shorouk City,Abu Al-Hool
25411,NaN,3,1,120,No,1,For Sale,Apartment,Cash,"6,000 EGP",شقه 120 متر بجاردينيا (1),New Cairo - El Tagamoa,Abu Al-Hool


In [77]:
df_all[df_all.Compound.str.contains('Gardenia',na=False)]

,Amenities,Bedrooms,Bathrooms,Area (m²),Furnished,Level,Ad Type,Type,Payment Option,price,title,location,Compound


In [78]:
# add gardinia data into entire data
df_all=df_all.append(df,ignore_index=True)

In [79]:
df_all[df_all.Compound=='Abu Al-Hool'].shape

(468, 13)

In [80]:
# check for duplication
df_all[df_all.duplicated()].shape[0]

4248

In [81]:
df_all.drop_duplicates(inplace=True)
df_all[df_all.duplicated()].shape[0]

0

In [82]:
df_all[df_all.Compound=='Abu Al-Hool'].shape

(389, 13)

In [83]:
# save all data
df_all.to_csv('./Data/olx_data_all_withGardinia.csv',index=False)